In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

คำย่อที่ควรรู้ก่อน
f คือ filter ,
a คือ no filter หรือ ใช้ raw ratings ,
n คือ no ,

Load ข้อมูล

In [2]:
df_f_ratings   = dataEngineering.load_Filter_Ratings()['data']
df_a_ratings        = dataEngineering.loadRatings()['data']
df_movies           = dataEngineering.loadMovies()['data']
df_users            = dataEngineering.loadUsers()['data']

สร้าง pivot table

In [3]:
df_f_index     = df_f_ratings.pivot(index='User_Index', columns='Movie_Index' , values='userRating')
df_fn_index = df_f_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

In [4]:
df_a_ratings["User_Index"] = df_a_ratings["UserID"].astype('category').cat.codes
df_a_ratings["Movie_Index"] = df_a_ratings["MovieID"].astype('category').cat.codes

In [5]:
df_a_index     = df_a_ratings.pivot(index='User_Index', columns='Movie_Index' , values='userRating')
df_an_index = df_a_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

learning_rate และ user_based ที่ดีที่สุด

In [6]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import GridSearchCV

# sim_options = {
#     'name': ['cosine', 'pearson'],
#     'user_based': [False, True],
# }
# bsl_options = {
#     'method': ['sgd'],
#     'learning_rate': [0.0005, 0.005]
# }
# param_grid = {
#     'k':[40]
#     'sim_options': sim_options,
#     'bsl_options': bsl_options
# }

# reader = Reader(rating_scale=(1, 5))
# data = Dataset.load_from_df(df_a_ratings[['UserID', 'MovieID', 'userRating']], reader)
# gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=5)
# gs.fit(data)

In [7]:
# gs.best_params

In [8]:
output = {
        'rmse': {
            'sim_options': {'name': 'pearson', 'user_based': True},
            'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}
    },
        'mae': {
            'sim_options': {'name': 'pearson', 'user_based': True},
            'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}
    }
}

In [9]:
options = {
    'sim_options': {'name': 'pearson', 'user_based': True},
    'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}
  }

In [10]:
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))

In [11]:
def create_KNNmodel(df, k_group):
    df = df.copy()
    print('k_group: ', k_group)
    data = Dataset.load_from_df(df.stack().reset_index(), reader)
    train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

    model = KNNWithMeans( 
        k=k_group,
        sim_options=options['sim_options'], 
        bsl_options=options['bsl_options']
    )
    model.fit(train_set)

    predictions = model.test(test_set)
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)
    return model

In [12]:
model_f_index = create_KNNmodel(df_f_index, 40)
model_fn_index = create_KNNmodel(df_fn_index, 40)

model_a_index = create_KNNmodel(df_a_index, 40)
model_an_index = create_KNNmodel(df_an_index, 40)

k_group:  40


Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8972
MAE:  0.6851
k_group:  40
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8972
MAE:  0.6851
k_group:  40
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9026
MAE:  0.6869
k_group:  40
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9026
MAE:  0.6869


ทดลองหา Models ที่ดีระหว่าง 4 Models ข้างต้น

In [13]:
# แบบมี Index
def predict_model_n_index(matrix, model):
    matrix = matrix.copy()
    u_col = 0
    ratings = []
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model.predict(u_id, m_id).est, 4)
                ratings.append([u_id, m_id, prep_rating])
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    
    recomment_unwatched = pd.DataFrame(ratings, columns=['UserID', 'MovieID', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)
# แบบไม่มี Index
def predict_model_index(matrix, model):
    matrix = matrix.copy()
    ratings = []
    for u_row, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            if pd.isnull(item):
                prep_rating = round(model.predict(u_row, m_col).est, 4)
                matrix.iloc[u_row][m_col] = prep_rating
                ratings.append([u_row, m_col, prep_rating])

    recomment_unwatched = pd.DataFrame(ratings, columns=['User_Index', 'Movie_Index', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

userID = 2

pivot_a_test  =  df_a_index.iloc[:userID].copy()
pivot_an_test = df_an_index.iloc[:userID].copy()

predict แบบ ไม่ใช้ Index

1. Models ได้ค่า Mean หรือก็คือไม่สามารถทำนาย unseen dataset ได้เลย

In [14]:
mat_fn, df_fn = predict_model_n_index(pivot_an_test, model_fn_index)
df_fn[df_fn["UserID"] == userID].head(5)

,UserID,MovieID,prep_Rating
15272,2,31702,3.4929
13380,2,5475,3.4929
15280,2,31921,3.4929
15267,2,31692,3.4929
15271,2,31700,3.4929


2. Models

In [15]:
mat_an, df_an = predict_model_n_index(pivot_an_test, model_an_index)
df_an[df_an["UserID"] == userID].head(5)

,UserID,MovieID,prep_Rating
14326,2,7247,5.0
15517,2,40412,5.0
13205,2,5147,5.0
18170,2,126548,5.0
17093,2,87867,5.0


predict แบบมีการใช้ Index

3. Models เหมือนจะดีแต่ User_Index ไม่ตรงกับ UserID ที่ต้องการ

In [16]:
mat_f, df_f = predict_model_index(pivot_a_test, model_f_index)
df_f[df_f["User_Index"] == userID-1].head(5)

,User_Index,Movie_Index,prep_Rating
14057,1,4569,5.0000
11621,1,2132,5.0000
13514,1,4026,5.0000
12805,1,3317,4.7680
10856,1,1367,4.5926


4. Models เหมือนกัน Models 2

In [17]:
mat_a, df_a = predict_model_index(pivot_a_test, model_a_index)
df_a[df_a["User_Index"] == userID-1].head(5)

,User_Index,Movie_Index,prep_Rating
14326,1,4839,5.0
15517,1,6031,5.0
13205,1,3717,5.0
18170,1,8706,5.0
17093,1,7619,5.0


Save Models

In [18]:
import joblib

joblib.dump(model_an_index, 'models/model_KNNwithMeans.pkl')

['models/model_KNNwithMeans.pkl']